# Environment Setup for Azure AI Foundry Workshop

This notebook will guide you through setting up your environment for the Azure AI Foundry workshop.

## Prerequisites
- Python 3.8 or later
- Azure subscription with AI services access
- Basic Python knowledge

In [ ]:
# Install required packages (if not already installed):
!pip install azure-identity azure-ai-projects

## Azure Authentication Setup
First, we'll verify our Azure credentials and setup.

In [1]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import ConnectionType
import os

# Initialize Azure credentials
try:
    credential = DefaultAzureCredential()
    print("✓ Successfully initialized DefaultAzureCredential")
except Exception as e:
    print(f"× Error initializing credentials: {str(e)}")

✓ Successfully initialized DefaultAzureCredential


## Initialize AI Project Client

> **Note:** Before proceeding, ensure you:
> 1. Copy your `.env.local` file to `.env`
> 2. Update the project connection string in your `.env` file
> 3. Have a Hub and Project already provisioned in Azure AI Foundry

You can find your project connection string in [Azure AI Foundry](https://ai.azure.com) under your project's settings:

<img src="proj-conn-string.png" alt="Project Connection String Location" width="600"/>



## Understanding AIProjectClient

The AIProjectClient is a key component for interacting with Azure AI services that:

- **Manages Connections**: Lists and accesses Azure AI resources like OpenAI models
- **Handles Authentication**: Securely connects using Azure credentials  
- **Enables Model Access**: Provides interfaces to use AI models and deployments
- **Manages Project Settings**: Controls configurations for your Azure AI project

The client requires:
- A project connection string (from Azure AI project settings)
- Valid Azure credentials

You can find your project connection string in Azure AI Studio under Project Settings:



In [2]:
from dotenv import load_dotenv
from pathlib import Path

# Load environment variables
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir / '.env')

# Initialize AIProjectClient with connection string and credentials
try:
    client = AIProjectClient(
        endpoint=os.getenv("PROJECT_CONNECTION_STRING"),
        credential=credential
    )
    print("✓ Successfully initialized AIProjectClient")
except Exception as e:
    print(f"× Error initializing client: {str(e)}")

✓ Successfully initialized AIProjectClient


## Verify Access to Models and Connections
Let's verify we can access all the required models and connections specified in the [prerequisites](../README.md#-prerequisites).

We need to validate:
- GPT models (gpt-4o, gpt-4o-mini) for chat/completion
- Embedding model for vector search 
- [Grounding with Bing](https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/bing-grounding?view=azure-python-preview&tabs=python&pivots=overview)
- Azure AI Search connection

This validation ensures we have all the components needed to build our AI applications.

In [8]:
# List the properties of all connections
connections = client.connections.list()
print(f"====> Listing of all connections:")
for connection in connections:
    print(connection)

# List the properties of all connections of a particular "type" (in this sample, Azure OpenAI connections)
connections = client.connections.list(
    connection_type=ConnectionType.AZURE_OPEN_AI,
)

====> Listing of all connections:
{'name': 'aaissc', 'id': '/subscriptions/611f3010-1b15-4826-b93d-8de76fcd6484/resourceGroups/AAI/providers/Microsoft.CognitiveServices/accounts/aaisc/projects/msft/connections/aaissc', 'type': 'CognitiveSearch', 'target': 'https://aaissc.search.windows.net/', 'isDefault': True, 'credentials': {'type': 'ApiKey'}, 'metadata': {'type': 'azure_ai_search', 'ApiType': 'Azure', 'ResourceId': '/subscriptions/611f3010-1b15-4826-b93d-8de76fcd6484/resourceGroups/AAI/providers/Microsoft.Search/searchServices/aaissc', 'ApiVersion': '2024-05-01-preview', 'DeploymentApiVersion': '2023-11-01'}}
{'name': 'bs', 'id': '/subscriptions/611f3010-1b15-4826-b93d-8de76fcd6484/resourceGroups/AAI/providers/Microsoft.CognitiveServices/accounts/aaisc/projects/msft/connections/bs', 'type': 'ApiKey', 'target': 'https://api.bing.microsoft.com/', 'isDefault': True, 'credentials': {'type': 'ApiKey'}, 'metadata': {'type': 'bing_grounding', 'ApiType': 'Azure', 'ResourceId': '/subscriptio

## Validate Model and Search Connections
The cell below validates that we have properly provisioned and connected to:
1. Azure OpenAI models through our Azure OpenAI connection
2. Azure AI Search through our Azure AI Search connection

Both of these services will be essential for building our AI applications. The OpenAI models will provide the core language capabilities, while Azure AI Search will enable efficient information retrieval and knowledge base functionality.



In [11]:
# List all connections and check for specific types
conn_list = client.connections.list()
search_conn_id = ""
openai_conn_id = ""

for conn in conn_list:
    print(conn)
    conn_type = str(conn.type).split('.')[-1]  # Get the part after the dot
    if conn_type == "AZURE_AI_SEARCH":
        search_conn_id = conn.id
    elif conn_type == "AZURE_OPEN_AI":
        openai_conn_id = conn.id

print(f"\n====> Connection IDs found:")
if not search_conn_id:
    print("Azure AI Search: Not found - Please create an Azure AI Search connection")
else:
    print(f"Azure AI Search: {search_conn_id}")
    
if not openai_conn_id:
    print("Azure OpenAI: Not found - Please create an Azure OpenAI connection") 
else:
    print(f"Azure OpenAI: {openai_conn_id}")

{'name': 'aaissc', 'id': '/subscriptions/611f3010-1b15-4826-b93d-8de76fcd6484/resourceGroups/AAI/providers/Microsoft.CognitiveServices/accounts/aaisc/projects/msft/connections/aaissc', 'type': 'CognitiveSearch', 'target': 'https://aaissc.search.windows.net/', 'isDefault': True, 'credentials': {'type': 'ApiKey'}, 'metadata': {'type': 'azure_ai_search', 'ApiType': 'Azure', 'ResourceId': '/subscriptions/611f3010-1b15-4826-b93d-8de76fcd6484/resourceGroups/AAI/providers/Microsoft.Search/searchServices/aaissc', 'ApiVersion': '2024-05-01-preview', 'DeploymentApiVersion': '2023-11-01'}}
{'name': 'bs', 'id': '/subscriptions/611f3010-1b15-4826-b93d-8de76fcd6484/resourceGroups/AAI/providers/Microsoft.CognitiveServices/accounts/aaisc/projects/msft/connections/bs', 'type': 'ApiKey', 'target': 'https://api.bing.microsoft.com/', 'isDefault': True, 'credentials': {'type': 'ApiKey'}, 'metadata': {'type': 'bing_grounding', 'ApiType': 'Azure', 'ResourceId': '/subscriptions/611f3010-1b15-4826-b93d-8de76fc